# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb

# Load Data

In [19]:
train = pd.read_csv(r".\train.csv")
test = pd.read_csv(r".\test.csv")

# Data Preprocessing

In [20]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


In [21]:
import pandas as pd
import re

# 아미노산 코드와 그들의 성질을 매핑
amino_acid_properties = {
    'A': 'nonpolar',    # Alanine
    'R': 'positive',    # Arginine
    'N': 'polar',       # Asparagine
    'D': 'negative',    # Aspartic Acid
    'C': 'polar',       # Cysteine
    'Q': 'polar',       # Glutamine
    'E': 'negative',    # Glutamic Acid
    'G': 'nonpolar',    # Glycine
    'H': 'positive',    # Histidine
    'I': 'nonpolar',    # Isoleucine
    'L': 'nonpolar',    # Leucine
    'K': 'positive',    # Lysine
    'M': 'nonpolar',    # Methionine
    'F': 'aromatic',    # Phenylalanine
    'P': 'nonpolar',    # Proline
    'S': 'polar',       # Serine
    'T': 'polar',       # Threonine
    'W': 'aromatic',    # Tryptophan
    'Y': 'aromatic',    # Tyrosine
    'V': 'nonpolar',    # Valine
}

# 변이 유형을 분류하는 함수 정의
def classify_mutation(mutation):
    # 결측치 처리
    if pd.isnull(mutation):
        return None  # 또는 특정 코드로 지정 가능

    mutation = str(mutation).strip()

    # WT 체크
    if mutation == 'WT':
        return 0  # WT (Wild Type)

    # 프레임시프트 돌연변이 체크 ('fs' 포함)
    if 'fs' in mutation:
        return 5  # 프레임시프트 돌연변이

    # 중단 돌연변이 체크 ('*' 포함)
    if '*' in mutation:
        return 4  # 중단 돌연변이

    # 돌연변이 패턴 매칭 (예: 'R496Q', 'L1700L')
    match = re.match(r'^([A-Z])(\d+)([A-Z])$', mutation)
    if match:
        from_aa = match.group(1)  # 원래 아미노산
        position = match.group(2) # 위치 (사용하지 않음)
        to_aa = match.group(3)    # 변이된 아미노산

        # 침묵 돌연변이 체크 (아미노산이 동일한 경우)
        if from_aa == to_aa:
            return 1  # 침묵 돌연변이

        # 아미노산 성질 가져오기
        from_property = amino_acid_properties.get(from_aa)
        to_property = amino_acid_properties.get(to_aa)

        # 아미노산 코드가 유효한지 확인
        if from_property is None or to_property is None:
            return 6  # 알 수 없는 아미노산 코드

        # 보존적 돌연변이 체크 (아미노산 성질이 동일한 경우)
        if from_property == to_property:
            return 2  # 보존적 돌연변이
        else:
            return 3  # 비보존적 돌연변이
    else:
        # 패턴 매칭 실패한 경우
        return 6  # 매칭 실패한 경우 6 반환

# 다중 치환을 처리하는 함수 정의
def classify_multiple_mutations(mutation_string):
    # 결측치 처리
    if pd.isnull(mutation_string):
        return None  # 또는 특정 코드로 지정 가능

    # 변이 문자열을 공백으로 분리
    mutations = str(mutation_string).strip().split()

    labels = []
    for mutation in mutations:
        label = classify_mutation(mutation)
        if label is not None:
            labels.append(label)
    if labels:
        # 가장 높은 값을 반환
        return max(labels)
    else:
        return None  # 또는 특정 코드로 지정 가능


In [22]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    train[col] = train[col].apply(classify_multiple_mutations)

# 결과 출력 (일부 열만 표시)
# print(train.head())

In [23]:
# print(train.head(19))

In [24]:
# 제외할 열 목록 (예시로 'ID'와 'SUBCLASS'를 제외)
exclude_cols = ['ID', 'SUBCLASS']

# 변이 데이터가 있는 열 목록
mutation_cols = [col for col in train.columns if col not in exclude_cols]

# 각 열에 함수 적용
for col in mutation_cols:
    test[col] = test[col].apply(classify_multiple_mutations)

# 결과 출력 (일부 열만 표시)
# print(test.head())

# Model Define and Train

LGBM 최적화

In [25]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report
# # import lightgbm as lgb
# import numpy as np
# from itertools import product
# from imblearn.over_sampling import SMOTE
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# from lightgbm import LGBMClassifier

# voting_clf = VotingClassifier(
#     estimators=[
#         ('lr',LogisticRegression()),
#         ('rf',RandomForestClassifier()),
#         ('svc',SVC()),
#         ('lgb',LGBMClassifier())
#     ]
# )

# # 1. 데이터 준비
# # 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
# X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS를 제외한 모든 열)
# y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# # 2. 학습 세트와 테스트 세트로 데이터 나누기
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# x_train_scaled = ss.fit_transform(X_train)
# x_test_scaled =  ss.fit_transform(X_test)

# # Apply SMOTE to balance the training data
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train)

# # Define a set of hyperparameters to iterate over for tuning
# learning_rates = [0.001, 0.05, 0.01, 0.1]
# num_leaves_options = [5, 10, 15]
# max_depths = [-1, 5, 10]  # -1 indicates no limit
# boosting_types = ['gbdt']  # 'dart', 'goss'

# # Create a list of all combinations of hyperparameters
# tuning_params = list(product(learning_rates, num_leaves_options, max_depths, boosting_types))

# best_train_accuracy = 0
# best_test_accuracy = 0
# best_params = None

# for lr, num_leaves, max_depth, boosting_type in tuning_params:
#     # Set parameters for the current iteration
#     params = {
#         'objective': 'multiclass',
#         'metric': 'multi_logloss',
#         'boosting_type': boosting_type,
#         'learning_rate': lr,
#         'num_leaves': num_leaves,
#         'max_depth': max_depth,
#         'verbose': -1,
#         'num_class': len(y.unique())
#     }
    
#     # Create and train the LightGBM model using LGBMClassifier
#     model = LGBMClassifier(**params)
#     model.fit(X_train_resampled, y_train_resampled)
    
#     # Make predictions on the train and test data
#     y_train_pred = model.predict(X_train_resampled)
#     y_test_pred = model.predict(X_test)
    
#     # Calculate accuracy
#     train_accuracy = accuracy_score(y_train_resampled, y_train_pred)
#     test_accuracy = accuracy_score(y_test, y_test_pred)
    
#     # Check if this is the best model so far
#     if test_accuracy > best_test_accuracy:
#         best_train_accuracy = train_accuracy
#         best_test_accuracy = test_accuracy
#         best_params = (lr, num_leaves, max_depth, boosting_type)
#         best_model = model  # Save the best model


- Voting
    - logisticregression
    - randomforestclassifier
    - svm.SVC
    - lightgbm
    - xgboost


In [26]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import VotingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# import lightgbm as lgb
# import xgboost as xgb
# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import SMOTE

# # 1. 데이터 준비
# # 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
# X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS를 제외한 모든 열)
# y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# # 2. 학습 세트와 테스트 세트로 데이터 나누기
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# # 3. 데이터 스케일링
# ss = StandardScaler()
# X_train_scaled = ss.fit_transform(X_train)
# X_test_scaled = ss.transform(X_test)  # Use transform instead of fit_transform on test set

# # 4. SMOTE 적용하여 학습 데이터 균형 맞추기
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)


# from tqdm import tqdm

# # 5. 개별 모델 정의
# models = [
#     ('lightgbm', lgb.LGBMClassifier(
#         objective='multiclass',
#         metric='multi_logloss',
#         boosting_type='gbdt',
#         learning_rate=0.1,
#         num_leaves=15,
#         max_depth=10,
#         verbose=-1,
#         num_class=len(y.unique())
#     )),
#     ('xgboost', xgb.XGBClassifier(
#         objective='multi:softprob',
#         eval_metric='mlogloss',
#         use_label_encoder=False,
#         learning_rate=0.1,
#         max_depth=10,
#         n_estimators=100
#     )),
#     ('logistic', LogisticRegression(max_iter=1000)),
#     ('svm', SVC(probability=True)),  # Enable probability estimates for voting
#     ('random_forest', RandomForestClassifier(n_estimators=100))
# ]

# # 6. 각 모델 학습
# for name, model in tqdm(models, desc="Training Models"):
#     model.fit(X_train_resampled, y_train_resampled)

# # 7. Voting Classifier 정의 (앙상블)
# voting_clf = VotingClassifier(
#     estimators=models,
#     voting='soft'  # Use soft voting to consider probabilities of each class
# )

# # 8. Voting Classifier 학습 및 평가
# voting_clf.fit(X_train_resampled, y_train_resampled)

# # 예측 및 평가
# y_pred = voting_clf.predict(X_test_scaled)
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))

In [27]:
# # Print the best results and parameters
# print(f"Best Train Accuracy: {best_train_accuracy * 100:.2f}%")
# print(f"Best Test Accuracy: {best_test_accuracy * 100:.2f}%")
# print(f"Best Parameters: Learning Rate={best_params[0]}, Num Leaves={best_params[1]}, Max Depth={best_params[2]}, Boosting Type={best_params[3]}")

# # Generate and print classification report for the best model on test data
# print("\nClassification Report:")
# print(classification_report(y_test, y_test_pred))

# # Use the best model to make predictions on the test set and store them
# predictions = best_model.predict(X_test)

# # Print or save predictions as needed
# print("Predictions on Test Set:")
# print(predictions)


모델 최적화 후 voting 진행

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from itertools import product
from tqdm import tqdm
from catboost import CatBoostClassifier

# 1. 데이터 준비
# 타겟: 'SUBCLASS', 특징: 'SUBCLASS'와 'ID'를 제외한 나머지 열
X = train.drop(columns=['SUBCLASS', 'ID'])  # 특징 데이터 (SUBCLASS를 제외한 모든 열)
y = train['SUBCLASS']  # 타겟 데이터 (SUBCLASS)

# 2. 학습 세트와 테스트 세트로 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 3. 데이터 스케일링
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)  # Use transform instead of fit_transform on test set

# 4. SMOTE 적용하여 학습 데이터 균형 맞추기
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Hyperparameter tuning for LightGBM with parallel processing
learning_rates = [0.001, 0.05, 0.01, 0.1]
num_leaves_options = [5, 10, 15]
max_depths = [-1, 5, 10]  # -1 indicates no limit
boosting_types = ['gbdt']  # 'dart', 'goss'

tuning_params = list(product(learning_rates, num_leaves_options, max_depths, boosting_types))

best_train_accuracy = 0
best_test_accuracy = 0
best_params = None

for lr, num_leaves, max_depth, boosting_type in tqdm(tuning_params, desc="Hyperparameter Tuning"):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting_type': boosting_type,
        'learning_rate': lr,
        'num_leaves': num_leaves,
        'max_depth': max_depth,
        'verbose': -1,
        'num_class': len(y.unique()),
        'n_jobs': -1  # Use all available cores for LightGBM training
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train_resampled, y_train_resampled)
    
    y_train_pred = model.predict(X_train_resampled)
    y_test_pred = model.predict(X_test_scaled)
    
    train_accuracy = accuracy_score(y_train_resampled, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    if test_accuracy > best_test_accuracy:
        best_train_accuracy = train_accuracy
        best_test_accuracy = test_accuracy
        best_params = (lr, num_leaves, max_depth, boosting_type)
        best_model = model

# Print the best results and parameters from hyperparameter tuning
print(f"Best Train Accuracy: {best_train_accuracy * 100:.2f}%")
print(f"Best Test Accuracy: {best_test_accuracy * 100:.2f}%")
print(f"Best Parameters: Learning Rate={best_params[0]}, Num Leaves={best_params[1]}, Max Depth={best_params[2]}, Boosting Type={best_params[3]}")

# Generate and print classification report for the best LightGBM model on test data
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Use the best LightGBM model to make predictions on the test set and store them
predictions_lgbm = best_model.predict(X_test_scaled)

# Print predictions from the best LightGBM model
print("Predictions on Test Set (LightGBM):")
print(predictions_lgbm)

# Define the CatBoost model with maximum CPU usage
catboost_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=10,
    loss_function='MultiClass',
    verbose=0,
    thread_count=-1  # Use all available cores for CatBoost training
)

# Ensemble Voting Classifier with the best LightGBM model included and parallel processing enabled
models = [
    ('lightgbm', best_model),  # Use the tuned LightGBM model here
    ('xgboost', xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='mlogloss',
        use_label_encoder=False,
        learning_rate=0.1,
        max_depth=10,
        n_estimators=100,
        n_jobs=-1)),  # Use all available cores for XGBoost training
    ('logistic', LogisticRegression(max_iter=1000)),
    ('svm', SVC(probability=True)),  # Enable probability estimates for voting; SVM does not support n_jobs directly in scikit-learn.
    ('random_forest', RandomForestClassifier(n_estimators=100, n_jobs=-1)),  # Use all available cores for Random Forest training
    ('catboost', catboost_model)  # Add the CatBoost model with maximum CPU usage enabled.
]

voting_clf = VotingClassifier(
    estimators=models,
    voting='soft',  # Use soft voting to consider probabilities of each class.
    n_jobs=-1       # Use all available cores for Voting Classifier fitting.
)

# Fit the Voting Classifier using all available CPU cores.
voting_clf.fit(X_train_resampled, y_train_resampled)

# Make predictions and evaluate the Voting Classifier.
y_pred_voting = voting_clf.predict(X_test_scaled)
print("Accuracy (Voting Classifier):", accuracy_score(y_test, y_pred_voting))
print("Classification Report (Voting Classifier):\n", classification_report(y_test, y_pred_voting))

Hyperparameter Tuning: 100%|██████████| 36/36 [20:04<00:00, 33.47s/it]


Best Train Accuracy: 87.79%
Best Test Accuracy: 34.57%
Best Parameters: Learning Rate=0.05, Num Leaves=10, Max Depth=-1, Boosting Type=gbdt

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.64      0.75        14
           1       0.25      0.05      0.08        21
           2       0.52      0.48      0.50       157
           3       0.19      0.10      0.13        31
           4       0.78      0.62      0.69        45
           5       1.00      0.14      0.25         7
           6       0.29      0.26      0.28        92
           7       0.44      0.36      0.40        45
           8       0.11      0.12      0.11       103
           9       0.13      0.16      0.14        67
          10       0.50      0.59      0.54        32
          11       0.17      0.17      0.17        46
          12       0.54      0.45      0.49        31
          13       0.12      0.03      0.04        37
          14       0.18  

In [29]:
# from catboost import CatBoostClassifier

# # Define the CatBoost model
# catboost_model = CatBoostClassifier(
#     iterations=100,
#     learning_rate=0.1,
#     depth=10,
#     loss_function='MultiClass',
#     verbose=0  # Set verbose to 0 to suppress output during fitting
# )

# # Add CatBoost to the list of models for the Voting Classifier
# models = [
#     ('lightgbm', best_model),  # Use the tuned LightGBM model here
#     ('xgboost', xgb.XGBClassifier(
#         objective='multi:softprob',
#         eval_metric='mlogloss',
#         use_label_encoder=False,
#         learning_rate=0.1,
#         max_depth=10,
#         n_estimators=100)),
#     ('logistic', LogisticRegression(max_iter=1000)),
#     ('svm', SVC(probability=True)),  # Enable probability estimates for voting
#     ('random_forest', RandomForestClassifier(n_estimators=100)),
#     ('catboost', catboost_model)  # Add the CatBoost model
# ]

# voting_clf = VotingClassifier(
#     estimators=models,
#     voting='soft'  # Use soft voting to consider probabilities of each class
# )

# # Fit the Voting Classifier
# voting_clf.fit(X_train_resampled, y_train_resampled)

# # Make predictions and evaluate the Voting Classifier
# y_pred_voting = voting_clf.predict(X_test_scaled)
# print("Accuracy (Voting Classifier):", accuracy_score(y_test, y_pred_voting))
# print("Classification Report (Voting Classifier):\n", classification_report(y_test, y_pred_voting))

In [30]:
model = best_model

# Inference

In [31]:
test = test.drop(columns=['ID'])
# X_encoded = test_X.copy()
# X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])

In [32]:
predictions = model.predict(test)

In [33]:
predictions

array([20, 25, 25, ..., 25, 25, 25])

In [34]:
original_labels = le_subclass.inverse_transform(predictions)

# Submisson

In [35]:
submisson = pd.read_csv("./sample_submission.csv")

In [36]:
submisson["SUBCLASS"] = original_labels

In [37]:
submisson.to_csv('./baseline_submission.csv', encoding='UTF-8-sig', index=False)